In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

In [51]:
file_path = "/Users/zerayacobmeshesha//Desktop/DevKit/Assignment 9 dataset-Faults.nna"

In [52]:
# Read the file as a list of lines
with open(file_path, "r") as file:
    lines = file.readlines()

# Split lines into rows and columns
data = [line.strip().split("\t") for line in lines]

# Convert to a DataFrame
import pandas as pd
data = pd.DataFrame(data)
print(data.head())

     0     1        2        3     4   5    6       7   8    9   ...      24  \
0    42    50   270900   270944   267  17   44   24220  76  108  ...  0.8182   
1   645   651  2538079  2538108   108  10   30   11397  84  123  ...  0.7931   
2   829   835  1553913  1553931    71   8   19    7972  99  125  ...  0.6667   
3   853   860   369370   369415   176  13   45   18996  99  126  ...  0.8444   
4  1289  1306   498078   498335  2409  60  260  246930  37  126  ...  0.9338   

        25      26 27 28 29 30 31 32 33  
0  -0.2913  0.5822  1  0  0  0  0  0  0  
1  -0.1756  0.2984  1  0  0  0  0  0  0  
2  -0.1228   0.215  1  0  0  0  0  0  0  
3  -0.1568  0.5212  1  0  0  0  0  0  0  
4  -0.1992       1  1  0  0  0  0  0  0  

[5 rows x 34 columns]


In [53]:
#Assign column names based on the assignment
dependent_columns = [
    "Pastry", "Z_Scratch", "K_Scratch", "Stains", "Dirtiness", "Bumps", "Other_Faults"]

In [54]:
independent_columns = [
    "X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter",
    "Y_Perimeter", "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity",
    "Length_of_Conveyor", "TypeOfSteel_A300", "TypeOfSteel_A400", "Steel_Plate_Thickness",
    "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index", "Edges_X_Index",
    "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index",
    "Orientation_Index", "Luminosity_Index", "SigmoidOfAreas"
]
data.columns = dependent_columns + independent_columns

In [55]:
# Combine dependent and independent variable names
data.columns = dependent_columns + independent_columns

# Split data into features (X) and targets (y)
X = data[independent_columns]
y = data[dependent_columns]

In [56]:
# Normalize the independent variables
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Process the dependent variables
# Combine fault categories into a single target variable (assumes one fault type per row)
y_combined = y.idxmax(axis=1)  # Identify the fault type with the highest value
y_encoded = pd.Categorical(y_combined).codes  # Convert to numeric codes
y_one_hot = to_categorical(y_encoded)  # Convert to one-hot encoding for classification

In [57]:
# Split dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_normalized, y_one_hot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [58]:
# Build the Neural Network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer matches number of fault types

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [59]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")

# Generate classification report and confusion matrix
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

print("Confusion Matrix:")
print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

Epoch 1/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2163 - loss: 1.7422 - val_accuracy: 0.3368 - val_loss: 1.6015
Epoch 2/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 540us/step - accuracy: 0.3476 - loss: 1.5646 - val_accuracy: 0.3608 - val_loss: 1.5407
Epoch 3/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - accuracy: 0.4173 - loss: 1.5091 - val_accuracy: 0.4055 - val_loss: 1.5061
Epoch 4/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step - accuracy: 0.4322 - loss: 1.4889 - val_accuracy: 0.4124 - val_loss: 1.4844
Epoch 5/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - accuracy: 0.4152 - loss: 1.4805 - val_accuracy: 0.4296 - val_loss: 1.4642
Epoch 6/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - accuracy: 0.4605 - loss: 1.4338 - val_accuracy: 0.4433 - val_loss: 1.4444
Epoch 7/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - accuracy: 0.4732 - loss: 1.4138 - val_accuracy: 0.4502 - val_loss: 1.4426
Epoch 8/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - accuracy: 0.4530 - loss: 1.4237 - val_accura